<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Let's-try-to-extract-the-list-of-bus-destinations" data-toc-modified-id="Let's-try-to-extract-the-list-of-bus-destinations-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Let's try to extract the list of bus destinations</a></span></li></ul></div>

# Beogradska autobuska stancija

### Let's try to extract the list of bus destinations

In [1]:
import xml.dom.minidom
doc = xml.dom.minidom.parse('destination.xml')
elements = doc.getElementsByTagName('DestinationDto')
for elem in elements:
    print(dir(elem))
    print (elem.firstChild.getElementsByTagNameNS)
    break

['ATTRIBUTE_NODE', 'CDATA_SECTION_NODE', 'COMMENT_NODE', 'DOCUMENT_FRAGMENT_NODE', 'DOCUMENT_NODE', 'DOCUMENT_TYPE_NODE', 'ELEMENT_NODE', 'ENTITY_NODE', 'ENTITY_REFERENCE_NODE', 'NOTATION_NODE', 'PROCESSING_INSTRUCTION_NODE', 'TEXT_NODE', '__bool__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_attrs', '_attrsNS', '_call_user_data_handler', '_child_node_types', '_ensure_attributes', '_get_attributes', '_get_childNodes', '_get_firstChild', '_get_lastChild', '_get_localName', '_get_tagName', '_localName', '_magic_id_nodes', 'appendChild', 'attributes', 'childNodes', 'cloneNode', 'firstChild', 'getAttribute', 'getAttributeNS', 'getAttributeNode', 'getAttri

In [2]:
!pip install lxml
from lxml import etree, objectify


In [3]:
with open('destination.xml', encoding="utf-8") as fobj:
    xml = fobj.read()
root = etree.fromstring(xml)
print(type(root))
print(root[0])
print(root.getchildren())

<class 'lxml.etree._Element'>
<Element {http://schemas.datacontract.org/2004/07/WebApiService.Models}result at 0x246ebcf7700>
[<Element {http://schemas.datacontract.org/2004/07/WebApiService.Models}result at 0x246ebcf7780>]


In [4]:
a = root.findall("{http://schemas.datacontract.org/2004/07/WebApiService.Models}result//{http://schemas.datacontract.org/2004/07/WebApiService.Models}DestinationDto")
destinations = {}
for elem in a:
    num = elem.getchildren()[0].text
    city = elem.getchildren()[1].text
    destinations[num] = city
print(len(destinations))

#it's not a mistake, this code add in a dictionnary the last value from duplicates of cities names, in сyrillic

2364


In [5]:
destinations

{'3': 'АДА  АС',
 '7': 'АЛЕКСАНДРОВАЦ АС',
 '9': 'АЛЕКСАНДРОВО НИШ',
 '10': 'АЛЕКСИНАЦ АС',
 '12': 'АЛЕКСИЋИ',
 '17': 'АПАТИН АС',
 '18': 'АРАНЂЕЛОВАЦ АС',
 '20': 'АРИЉЕ АС',
 '22': 'БАРИЧ ГОЛУБАЦ',
 '24': 'БАБУШНИЦА АС',
 '26': 'БАВАНИШТЕ  Ц',
 '28': 'БАДОВИНЦИ',
 '30': 'МАЛА ВРБНИЦА',
 '32': 'ГРАДАЦ СЈЕНИЦА',
 '37': 'БАЈИНА БАШТА АС',
 '38': 'БАЈМОК',
 '39': 'БАНАТСКО ВЕЛИКО СЕЛО',
 '40': 'БАНАТСКИ ДВОРОВИ',
 '41': 'БАНАТСКО КАРАЂОРЂЕВО',
 '42': 'БАНАТСКИ КАРЛОВАЦ',
 '46': 'БАНСТОЛ',
 '48': 'МАЛО ГОЛУБИЊЕ',
 '49': 'БАЊА ВРУЈЦИ',
 '51': 'БАЊА КОВИЉАЧА',
 '52': 'БАЊАНИ',
 '55': 'БАЊА КОВИЉАЧА МОТЕЛ',
 '56': 'КОНАРЕВО',
 '58': 'СКУПЉЕН БАРАКЕ',
 '59': 'ДОЛИНА ЈОРГОВАНА',
 '60': 'БАРЕ',
 '62': 'БАРИЦЕ',
 '63': 'БАРЛОВО',
 '64': 'БАРОШЕВАЦ',
 '67': 'БАТАЈНИЦА',
 '68': 'БОРЧА Р',
 '70': 'БАТОЧИНА АС',
 '71': 'БАТУША',
 '72': 'БАЧ АС',
 '73': 'БАЧИНА',
 '74': 'БАЧИНЦИ',
 '75': 'БАЧКО ГРАДИШТЕ',
 '76': 'БАЧКИ ЈАРАК',
 '77': 'ДОЈЕВИЦЕ',
 '78': 'БАЧЕВЦИ',
 '79': 'БАЧКА ПАЛАНКА АС',
 '80': 'БАЧ

Done! We've got a list of bus destinations from Belgrade